# Trabalho Final - Aprendizado de Maquina (Parte 01)

IFMG - Instituto Federal de Educação, Ciência e Tecnologia de Minas Gerais - Especialização em Inteligência Artificial  

Mat.: Aprendizado de Máquina 

Prof.: Frederico Gadelha Guimarães

Aluno: Christhian da S. Gonçalves 

## 01 - Base de Dados

Conjunto de dados 01: Dados reais de manutenção preditiva são geralmente difíceis de obter, então foi gerado um conjunto de dados sintético que reflete a manutenção preditiva real encontrada na indústria.

O conjunto de dados consiste em 10.000 pontos de dados armazenados como linhas com 14 recursos em colunas UID:  A falha da máquina consiste em cinco modos de falha independentes, falha de desgaste da ferramenta (TWF): a ferramenta será substituída ou falha em um tempo de desgaste da ferramenta selecionado aleatoriamente entre 200 - 240 minutos (120 vezes em nosso conjunto de dados). Neste momento, a ferramenta é substituída 69 vezes e falha 51 vezes (atribuídas aleatoriamente). falha de dissipação de calor (HDF): a dissipação de calor causa uma falha no processo, se a diferença entre a temperatura do ar e do processo for inferior a 8,6 K e a velocidade de rotação da ferramenta for inferior a 1380 rpm. Este é o caso de 115 pontos de dados. falha de energia (PWF): o produto do torque e da velocidade de rotação (em rad/s) é igual à potência necessária para o processo. Se esta potência estiver abaixo de 3500 W ou acima de 9000 W, o processo falha, o que ocorre 95 vezes em nosso conjunto de dados. falha de sobretensão (OSF): se o produto do desgaste da ferramenta e do torque exceder 11.000 minNm para a variante de produto L (12.000 M, 13.000 H), o processo falha devido à sobretensão. Isso é verdade para 98 pontos de dados. falhas aleatórias (RNF): cada processo tem uma chance de 0,1% de falhar independentemente de seus parâmetros de processo. Este é o caso de apenas 5 pontos de dados, menos do que o esperado para 10.000 pontos de dados em nosso conjunto de dados. Se pelo menos um dos modos de falha acima for verdadeiro, o processo falha e o rótulo 'falha da máquina' é definido como 1. Portanto, não é transparente para o método de aprendizado de máquina.

Link: https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset